# SC13 Team 8

In [93]:
# Basic Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
sb.set() # set the default Seaborn style for graphics

# Practical Motivation

Kickstarter is a corporation that maintains a crowdfunding platform for creative projects. To date, over USD 6 Billion dollars have been pledged to kickstarter projects with over 200,000 successfully funded projects. However, over 300,000 projects are unsuccessful in funding.

How can kickstarter campaign starters reduce their chances of failing?

Problem definition: How do different variables affect the failure of kickstarter campaigns?

# Data Extraction

Since Kickstarter does not have a public api, we are not going to scrape off their website using available tools. Instead, we will use the 2018 dataset from Kaggle.

In [94]:
ksData = pd.read_csv('ks-projects-201801.csv')

In [95]:
ksData.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


In [96]:
ksData.shape

(378661, 15)

The 2018 dataset has a size of 378661 x 15.

# Data Cleaning and Preparation

First, let us explore the different variables we're working with to make sure only relevant data is kept.

In [97]:
for col in ksData.columns:
    print(col)

ID
name
category
main_category
currency
deadline
goal
launched
pledged
state
backers
country
usd pledged
usd_pledged_real
usd_goal_real


Since we are exploring how different variables affect the failure of kickstarter campaigns, we will first remove the variables that do not affect the failure at all.

These variables include:

ID - since ID is a unique tag given by Kickstarter, it does not have any meaning

currency - whether it is in USD or GBP, the value is equal when converted, it does not matter.

country- since Kickstarter is a global platform, although success rates may vary by country, changing the country to reduce the failure rates is not feasible.

In [98]:
ksData.drop(columns=['ID','currency','country'], inplace = True)

Next, we remove any variables that are tied to our response variable "state" since they do not give additional insight.

These variables include: pledged, backers, usd pledged and usd_pledged_real.

In [99]:
ksData.drop(columns=['pledged', 'backers', 'usd pledged', 'usd_pledged_real'], inplace = True)

Since we are already using main_category, we can exclude category as a variable. Same case for goal and usd_goal_real

In [100]:
ksData.drop(columns=['category', 'goal'], inplace = True)

In [101]:
for col in ksData.columns:
    print(col)

name
main_category
deadline
launched
state
usd_goal_real


We are left with the following predictor variables: name, main_category, deadline, launched

and our response variable is 'state'.

Before we turn the predictor variables into more meaningful data, we will remove any null values from the dataset.

In [102]:
ksData.isnull().sum()

name             4
main_category    0
deadline         0
launched         0
state            0
usd_goal_real    0
dtype: int64

In [103]:
ksData.dropna(inplace=True)

In [104]:
ksData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 378657 entries, 0 to 378660
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   name           378657 non-null  object 
 1   main_category  378657 non-null  object 
 2   deadline       378657 non-null  object 
 3   launched       378657 non-null  object 
 4   state          378657 non-null  object 
 5   usd_goal_real  378657 non-null  float64
dtypes: float64(1), object(5)
memory usage: 20.2+ MB


Now that we have confirmed that we have no missing values in our dataset, we can proceed to prepare the predictor variables.

In [105]:
ksData['name']

0                           The Songs of Adelaide & Abullah
1             Greeting From Earth: ZGAC Arts Capsule For ET
2                                            Where is Hank?
3         ToshiCapital Rekordz Needs Help to Complete Album
4         Community Film Project: The Art of Neighborhoo...
                                ...                        
378656    ChknTruk Nationwide Charity Drive 2014 (Canceled)
378657                                            The Tribe
378658    Walls of Remedy- New lesbian Romantic Comedy f...
378659                             BioDefense Education Kit
378660                    Nou Renmen Ayiti!  We Love Haiti!
Name: name, Length: 378657, dtype: object

The 'name' is just a string of words, we need to turn this into more meaningful data to be used in our machine learning model.

We will create a new column called 'name_length', which is the length of the name in characters. This will be used in our machine learning model. 'name' will be kept for later text visualisation purposes in EDA.


In [106]:
ksData['name_length'] = ksData['name'].str.len()

Similarly, 'launched' and 'deadline' itself does not help us decipher any useful information. Instead, we can consider the campaign length, computed using deadline-launched. A measure of campaign length will help us understand the appropriate or inappropriate length for campaigns.

'campaign_length' will be measured in days for better rounding of data.

In [107]:
ksData['launched'] = pd.to_datetime(ksData['launched'])
ksData['launched_date'] = ksData['launched'].dt.date

ksData['deadline'] = pd.to_datetime(ksData['deadline'])
ksData['deadline_date'] = ksData['deadline'].dt.date

ksData['campaign_length'] = (ksData['deadline_date'] - ksData['launched_date']).dt.days
ksData['campaign_length'] = ksData['campaign_length'].astype(int)


We can remove the launched and dateline columns afterwards.

In [108]:
ksData.drop(columns=['launched', 'deadline', 'launched_date', 'deadline_date'], inplace = True)

-PROGRESS CHECK-
Take a look at what the dataset looks like now

In [109]:
ksData

,name,main_category,state,usd_goal_real,name_length,campaign_length
0,The Songs of Adelaide & Abullah,Publishing,failed,1533.95,31,59
1,Greeting From Earth: ZGAC Arts Capsule For ET,Film & Video,failed,30000.00,45,60
2,Where is Hank?,Film & Video,failed,45000.00,14,45
3,ToshiCapital Rekordz Needs Help to Complete Album,Music,failed,5000.00,49,30
4,Community Film Project: The Art of Neighborhoo...,Film & Video,canceled,19500.00,58,56
...,...,...,...,...,...,...
378656,ChknTruk Nationwide Charity Drive 2014 (Canceled),Film & Video,canceled,50000.00,49,30
378657,The Tribe,Film & Video,failed,1500.00,9,27
378658,Walls of Remedy- New lesbian Romantic Comedy f...,Film & Video,failed,15000.00,71,46
378659,BioDefense Education Kit,Technology,failed,15000.00,24,31


Next, we will turn 'main_category' into numerical data through one-hot encoding for the different types of categories.
We avoid using label encoding here to avoid any misunderstanding that one category is better than the other.

In [110]:
one_hot_data = pd.get_dummies(ksData['main_category'])

In [111]:
one_hot_data

,Art,Comics,Crafts,Dance,Design,Fashion,Film & Video,Food,Games,Journalism,Music,Photography,Publishing,Technology,Theater
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378656,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
378657,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
378658,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
378659,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


Lastly, we will look at our response variable 'state'

In [112]:
ksData['state'].value_counts()

failed        197716
successful    133956
canceled       38779
undefined       3562
live            2799
suspended       1845
Name: state, dtype: int64

Since we want to test the variables against the failed state, we only want successful and failed.


"live" and "undefined" will be removed, "canceled" and "suspended" will be added to failed.

In [116]:
ksData = ksData[ksData["state"].str.contains("live") == False]
ksData = ksData[ksData["state"].str.contains("undefined") == False]

In [117]:
ksData['state'] = ksData['state'].replace({'canceled':'failed', 'suspended':'failed'})

In [118]:
ksData['state'].value_counts()

failed        238340
successful    133956
Name: state, dtype: int64

We can spot an obvious class imbalance here. 
Since the dataset is sufficiently large, we will undersample the data instead of oversampling it.

In [119]:
from sklearn.utils import resample

ksData[ksData['state'].str.contains('failed')] = resample(ksData[ksData['state'].str.contains('failed')], replace = False, n_samples = len(ksData[ksData['state'].str.contains('successful')]), random_state = 20)

ksData['state'].value_counts()

failed        133956
successful    133956
Name: state, dtype: int64